In [1]:
import os 
from typing import List, Dict, Any
import pandas as pd

In [16]:
from langchain_core.documents import Document
from langchain_text_splitters import (RecursiveCharacterTextSplitter,
                                     CharacterTextSplitter,
                                     TokenTextSplitter)
print("Text Splitters Loaded")
print("SetUp Completed!")

Text Splitters Loaded
SetUp Completed!


## Understanding Doc structure In LangChain

In [18]:
## create a simple doc
doc = Document(
    page_content="This is the main text content that will be embedde and serached",
    metadata = {
        "source" : "example.txt",
        "page" : 1,
        "author" : "Ayman El Ouardiji",
        "data_created" : "2025-11-11"
    }
)
print("Doc structure")
print(f"Content : {doc.page_content}")
print(f"metadata : {doc.metadata}")

Doc structure
Content : This is the main text content that will be embedde and serached
metadata : {'source': 'example.txt', 'page': 1, 'author': 'Ayman El Ouardiji', 'data_created': '2025-11-11'}


### Text files .txt - Simplest Case

In [19]:
## Create a simple text file 
import os
os.makedirs("data/text_files", exist_ok=True)

In [ ]:
sample_texts={
    "data/text_files/python_intro.txt": """
    Python is a high-level, interpreted programming language known for its simplicity and readability. Created by Guido van Rossum and first released in 1991, Python has become one of the most popular programming languages in the world.

    Key Features:

    Easy to learn and use

    Extensive standard library

    Cross-platform compatibility

    Strong community support

    Python is widely used in web development, data science, artificial intelligence, and automation.""",
    "data/text_files/machine_learning.txt": """Machine Learning Basics

    Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn for themselves.

    Types of Machine Learning:

    Supervised Learning: Learning with labeled data

    Unsupervised Learning: Finding patterns in unlabeled data

    Reinforcement Learning: Learning through rewards and penalties

    Applications include image recognition, speech processing, and recommendation systems
    """
}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)
print("Sample text files created!")

Sample text files created!
Sample text files created!


### TextLoader- Read Single File

In [6]:
#from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

## Loading a single text file
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()
#print(type(documents))
#print(documents)
print(f"Loaded {len(documents)} document")
print(f"Content preview : {documents[0].page_content[:100]}...")
print(f"Metadata: {documents[0].metadata}")


Loaded 1 document
Content preview : 
    Python is a high-level, interpreted programming language known for its simplicity and readabili...
Metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader- Multiple text files

In [7]:
from langchain_community.document_loaders import DirectoryLoader
## Loading multiple text files from a directory
dir_loader = DirectoryLoader("data/text_files"
                             , glob="**/*.txt" #Pattern to match files
                             ,loader_cls = TextLoader,
                              loader_kwargs={'encoding' : 'utf-8'},
                              show_progress=True)

documents = dir_loader.load()
print(f"Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
    print(f"\nDocuments {i+1}:")
    print (f" Source: {doc.metadata['source']}")
    print(f" Length: {len(doc.page_content)} characters")



100%|██████████| 2/2 [00:00<00:00, 355.34it/s]

Loaded 2 documents

Documents 1:
 Source: data\text_files\machine_learning.txt
 Length: 591 characters

Documents 2:
 Source: data\text_files\python_intro.txt
 Length: 481 characters


### TEXT Splitting Strategies

In [14]:
## Different Text Splitters strategies
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
) 

print(documents)

[Document(metadata={'source': 'data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn for themselves.\n\n    Types of Machine Learning:\n\n    Supervised Learning: Learning with labeled data\n\n    Unsupervised Learning: Finding patterns in unlabeled data\n\n    Reinforcement Learning: Learning through rewards and penalties\n\n    Applications include image recognition, speech processing, and recommendation systems\n    '), Document(metadata={'source': 'data\\text_files\\python_intro.txt'}, page_content='\n    Python is a high-level, interpreted programming language known for its simplicity and readability. Created by Guido van Rossum and first released in 1991, Python has become one of the most popular programming language

In [15]:
### Method 1- Character Text Splitter

text = documents[0].page_content
text

'Machine Learning Basics\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn for themselves.\n\n    Types of Machine Learning:\n\n    Supervised Learning: Learning with labeled data\n\n    Unsupervised Learning: Finding patterns in unlabeled data\n\n    Reinforcement Learning: Learning through rewards and penalties\n\n    Applications include image recognition, speech processing, and recommendation systems\n    '

In [19]:
### Method 1- Character Text Splitter
print("Character Text Splitter")
char_splitter = CharacterTextSplitter(
    separator="\n", #Split on newlines
    chunk_size=200,# Max chunk size in characters
    chunk_overlap=20, #Overlap between chunks
    length_function=len #How to measure chunk size
)

char_chunk = char_splitter.split_text(text)
print(f"Created {len(char_chunk)} chunks")
print(f"First chunk: {char_chunk[0][:100]}...")

Created a chunk of size 254, which is longer than the specified 200


Character Text Splitter
Created 4 chunks
First chunk: Machine Learning Basics...


In [25]:
print(char_chunk[0])
print("---------------")
print(char_chunk[1])
print("---------------")
print(char_chunk[2])
print("---------------")
print(char_chunk[3])



Machine Learning Basics
---------------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn for themselves.
---------------
Types of Machine Learning:
    Supervised Learning: Learning with labeled data
    Unsupervised Learning: Finding patterns in unlabeled data
---------------
Reinforcement Learning: Learning through rewards and penalties
    Applications include image recognition, speech processing, and recommendation systems


In [37]:
### Method 2- Recursive Character Splitter
print("RECURSIVE CHARACTER TEXT SPLITTER")
recursive_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " ", ""], # try these separators in order
    chunk_size=200, # Max chunk size in characters
    chunk_overlap=20, # Overlap between chunks
    length_function=len # How to measure chunk size
)

recursive_chunk = recursive_splitter.split_text(text)
print(f"Created {len(recursive_chunk)} chunks")
print(f"First chunk: {recursive_chunk[0][:100]}...")

RECURSIVE CHARACTER TEXT SPLITTER
Created 5 chunks
First chunk: Machine Learning Basics...


In [38]:
print(recursive_chunk[0])
print("---------------")
print(recursive_chunk[1])
print("---------------")
print(recursive_chunk[2])
print("---------------")
print(recursive_chunk[3])

Machine Learning Basics
---------------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs
---------------
computer programs that can access data and use it to learn for themselves.
---------------
Types of Machine Learning:

    Supervised Learning: Learning with labeled data

    Unsupervised Learning: Finding patterns in unlabeled data


In [39]:
# Create text without natural break points
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence thre"

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],   # Only split on spaces
    chunk_size=80,
    chunk_overlap=20,
    length_function=len
)

chunks = splitter.split_text(simple_text)

print(f"\nSimple text example - {len(chunks)} chunks:\n")

for i in range(len(chunks) - 1):
    print(f"Chunk {i+1}: '{chunks[i]}'")
    print(f"Chunk {i+2}: '{chunks[i+1]}'")

    print()



Simple text example - 2 chunks:

Chunk 1: 'This is sentence one and it is quite long. This is sentence two and it is also'
Chunk 2: 'two and it is also quite long. This is sentence thre'



In [40]:
### Method 2- Token-based splitting
print("TOKEN TEXT SPLITTER")
token_splitter = TokenTextSplitter(
    chunk_size=50, # Max chunk size in characters
    chunk_overlap=10, # Overlap between chunks
)

token_chunk = token_splitter.split_text(text)
print(f"Created {len(token_chunk)} chunks")
print(f"First chunk: {token_chunk[0][:100]}...")

TOKEN TEXT SPLITTER
Created 3 chunks
First chunk: Machine Learning Basics

    Machine learning is a subset of artificial intelligence that enables sy...


In [41]:
# 📊 Comparison
print("\n📊 Text Splitting Methods Comparison:")

print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  👉 Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  👉 Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  👉 Use when: Working with token-limited models")



📊 Text Splitting Methods Comparison:

CharacterTextSplitter:
  ✅ Simple and predictable
  ✅ Good for structured text
  ❌ May break mid-sentence
  👉 Use when: Text has clear delimiters

RecursiveCharacterTextSplitter:
  ✅ Respects text structure
  ✅ Tries multiple separators
  ✅ Best general-purpose splitter
  ❌ Slightly more complex
  👉 Use when: Default choice for most texts

TokenTextSplitter:
  ✅ Respects model token limits
  ✅ More accurate for embeddings
  ❌ Slower than character-based
  👉 Use when: Working with token-limited models
